# Minneapolis Purple Air Sensor Project
## Elevation data wrangling ETL and QAQC

In [ ]:
import arcpy # use arc pro objects and functions
from arcpy import env # to set working environmnet
import pandas as pd # manipulate data
import requests #fetch data from apis
import zipfile #compressed data
import io #data decoding
import numpy as np #make arrays out of rasters
import psycopg2 # manage sql and db connection

### Set up environment, define functions

In [ ]:
#set project directory path
proj_dir = r'C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc'
#set geodatabase path
proj_gdb = r'C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb'
#set arcpy environment, this is where this code will store the imported data
arcpy.env.workspace = proj_dir
# allow files to be overwritten
arcpy.env.overwriteOutput = True 
#test
arcpy.env.workspace

In [ ]:
# set up fnction to unzip requests and save to the project's working directory
def unzip(input_zipped, working_dir):
    get_zipped = zipfile.ZipFile(
        io.BytesIO(
            input_zipped.content)
    )
    
    get_zipped.extractall(working_dir)

### Get the data

In [33]:
# set data url endpoint
base = r'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/projects/metro/block_metro/geodatabase/'
gdb_list = [r'3542-31-22.gdb.zip', 
            r'3542-31-23.gdb.zip',
           ]

In [ ]:
# fetch the data 
for gdb in gdb_list:
    response = requests.get(base + gdb)
    unzip(response, proj_dir)

### Merge the raster tiles for consistancy in analysis

In [36]:
# create a list of the new raster file names from the imported data
file_list_new = gdb_list
in_rasters = []

for i in range(len(file_list_new)):
    file_list_new[i] = file_list_new[i].replace(".zip", "") + "\dem_1m_m"
    
    # Create a new variable for each file using exec()
    exec(f'var_{i} = "{file_list_new[i]}"')
    
    # add that variable to the input rasters for the merge
    in_rasters.append(eval(f'var_{i}'))

In [37]:
in_rasters

['3542-31-22.gdb\\dem_1m_m', '3542-31-23.gdb\\dem_1m_m']

In [38]:
# merge raster tiles
from arcpy.ia import * # for merging rasters

# Set input rasters
# in_raster1 = arcpy.Raster(r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\3542-31-22.gdb\dem_1m_m")
# in_raster2 = arcpy.Raster(r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\3542-31-23.gdb\dem_1m_m")
# #in_raster3 = arcpy.Raster("")

# Merge the rasters
mpls_metro_elev = arcpy.ia.Merge(in_rasters, "FIRST")
mpls_metro_elev.save("mpls_metro_elev.tif")

### Clip to Extent

In [24]:
# define extent of study (2km buffer around Mpls)
# Minneapolis Boundary (from MetCouncil CTUs and County Boundaries)
import os

# get boundary
mpls_bndry_path = os.path.join(os.getcwd(), 'mpls_boundary.geojson')
#TODO: Make this an api call to auto download the file if not present in cwd


# Specify the input GeoJSON file path
input_geojson = mpls_bndry_path
# Specify the output feature class path and name
output_fc = proj_gdb + r'\mpls_bndry'

# Convert the GeoJSON to features
arcpy.JSONToFeatures_conversion(input_geojson, output_fc)


<Result 'C:\\Users\\MrJDF\\Desktop\\arc2_lab2\\mpls_prplair_qaqc\\mpls_prplair_qaqc.gdb\\mpls_bndry'>

In [26]:
#buffer

# Specify the input feature class path and name
input_fc = output_fc
# Specify the output feature class path and name for the buffer
output_buffer = proj_gdb + r'\mpls_2km_buffer'

# Buffer the input feature class to 2km
arcpy.Buffer_analysis(input_fc, output_buffer, "2000 Meters")

<Result 'C:\\Users\\MrJDF\\Desktop\\arc2_lab2\\mpls_prplair_qaqc\\mpls_prplair_qaqc.gdb\\mpls_2km_buffer'>

In [29]:
# clip
import arcpy

# Set the input and output file paths
input_raster = r'mpls_metro_elev.tif'
output_raster = r'aoi_elev.tif'

#set extent
clipping_extent = output_buffer

# Clip the input raster to the clipping extent
arcpy.Clip_management(input_raster, clipping_extent, output_raster, "#", "#", "NONE")

<Result 'C:\\Users\\MrJDF\\Desktop\\arc2_lab2\\mpls_prplair_qaqc\\aoi_elev.tif'>

### Convert the raster to a numpy array

- note
    - -3.402823e+38 is the default "no data/nan" value in arcpro

In [3]:
import numpy as np #make arrays out of rasters

In [4]:
# Set the input raster path
input_raster = r'aoi_elev.tif'

# Convert the raster to a numpy array
raster_array = arcpy.RasterToNumPyArray(input_raster)

# Print the shape of the array
print(raster_array.shape)

# Print the minimum and maximum values in the array
print(np.min(raster_array))
print(np.max(raster_array))


(3593, 5038)
-3.402823e+38
272.025


### raster stats from the numpy array

- things to look for
    - resolution: is it consistent across the data set?
    - Extent : 
        - all xy positions are withing bounds of aoi
        - no gaps or overlaps
    - No data values: how are these handled?
    - distribution: is it normal and consistant?

In [6]:
data = raster_array

min_val = np.min(data)
max_val = np.max(data)
#mean_val = np.mean(data) # too large for arcpro envi?
std_val = np.std(data)
median_val = np.median(data)
percentile_90 = np.percentile(data, 90)
variance_val = np.var(data)

In [8]:
import seaborn as sns
sns.distplot(data.flatten(), kde=False, bins=50)
plt.axvline(np.mean(data), color='red', linestyle='dashed', linewidth=2)
plt.axvline(np.mean(data) + np.std(data), color='black', linestyle='dashed', linewidth=2)
plt.axvline(np.mean(data) - np.std(data), color='black', linestyle='dashed', linewidth=2)
plt.show()


ModuleNotFoundError: No module named 'seaborn'

In [43]:
# convert to a pandas dataframe 

df = pd.DataFrame(raster_array)

In [45]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4998,4999,5000,5001,5002,5003,5004,5005,5006,5007,5008,5009,5010,5011,5012,5013,5014,5015,5016,5017,5018,5019,5020,5021,5022,5023,5024,5025,5026,5027,5028,5029,5030,5031,5032,5033,5034,5035,5036,5037
0,263.214386,263.196503,263.297607,263.269104,263.317505,263.393311,263.336914,263.211212,263.139404,263.140503,263.098206,263.105286,263.124786,263.093414,263.086212,263.047211,263.053101,263.076111,263.034607,263.028290,263.028687,263.070007,263.052094,263.075714,263.051910,263.036987,263.000000,263.007812,263.026215,262.992798,263.041595,263.040314,263.045593,263.042999,263.027710,262.996796,262.980591,262.936493,262.935394,262.928986,...,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38
1,263.244690,263.261108,263.282410,263.291107,263.274994,263.353088,263.291595,263.134888,263.116089,263.092896,263.112396,263.062286,263.109192,263.106903,263.078705,263.053894,263.061798,263.048798,263.050293,263.029114,263.031799,263.036102,263.085785,263.082214,263.042297,263.045013,263.028290,262.979401,262.977905,262.982605,263.019989,262.998810,263.038788,263.012909,263.041595,263.024414,262.971008,262.961090,262.946014,262.892090,...,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38
2,263.236694,263.263000,263.291412,263.258301,263.259186,263.301697,263.277191,263.181702,263.119202,263.096802,263.086609,263.100311,263.115814,263.107788,263.078491,263.013306,263.019989,263.048401,263.034393,263.028107,263.048187,263.035706,263.059998,263.069092,263.073090,263.048096,263.032990,262.988098,263.003815,263.007294,263.040497,263.036407,263.014191,263.032898,263.045197,263.019714,262.971985,263.030212,262.992096,262.945007,...,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38
3,263.174713,263.218414,263.322601,263.275085,263.308289,263.295502,263.284485,263.211792,263.147400,263.148712,263.100006,263.124115,263.151001,263.059998,263.042908,263.049103,263.026703,263.075104,263.030792,263.079102,263.071899,263.085205,263.029694,263.048004,263.052002,263.047394,263.029694,262.971893,263.019989,263.068604,263.095001,263.060211,262.997711,263.011902,263.015594,263.026794,262.991486,263.036713,263.035095,262.970398,...,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3593 entries, 0 to 3592
Columns: 5038 entries, 0 to 5037
dtypes: float32(5038)
memory usage: 69.1 MB


## Create SQL tables in remote DB from raster data

In [ ]:
# # Open a connection to the PostgreSQL database
# conn = psycopg2.connect(database="mydatabase", user="myusername", password="mypassword", host="localhost", port="5432")

# # Create a cursor object
# cur = conn.cursor()

# # Create the SQL table
# cur.execute("CREATE TABLE mytable (col1 INTEGER, col2 INTEGER, col3 INTEGER)")

# # Insert the NumPy array into the SQL table
# for row in arr:
#     cur.execute("INSERT INTO mytable (col1, col2, col3) VALUES (%s, %s, %s)", tuple(row))

# # Commit the changes to the database
# conn.commit()

# # Close the cursor and connection
# cur.close()
# conn.close()


In [ ]:
# command line tool for loading raster data to sql database
#TODO: convert the raster file to a GEOtiff?
# raster2pgsql -s <source_srid> -I -C -M <raster_file> <table_name> | psql -h <host> -d <database> -U <username>
